### 시계열 데이터 분석 with Pandas

참고 : https://github.com/lovablebaby/DataScience/blob/master/%5BPython%5DpyconTutorial_2018/.ipynb_checkpoints/timeSeriesAnalysis-checkpoint.ipynb

- Time series with pandas
- 시계열 데이터 특성
- ETS 모델
- EWMA 모델

시계열 데이터는 시간 데이터를 인덱스로 하는 연속된 데이터임. 데이터 자체를 DateTime타입으로 바꿔서 작업하게 됨.

- DateTime : 시간이나 날짜 정보는 인덱스인 경우가 많고, 통일도 잘 안되있는 경우가 더러 있음. 판다스에 내장된 기능들로 DateTime으로 다룸

#### time resampling
시계열 데이터 인덱스는 작은 단위로도 이루어진 경우도 있음. 더 넓은 주기로 집계하기 위해서 Time Resampling 이 필요함.

Groupby로도 가능하지만 비즈니스 도메인에서 분기나 회계년도를 편리하게 처리할 떄는 그런 방법은 안 된다. 판다스는 이런 경우에 활용할 수 있는 프리퀀시 샘플링 도구를 지원.

나라마다 주말이 다를 수도 있기 때문에 계산할 때도 타임 리샘플링을 하면 편리하게 사용할 수 있음

#### Rolling and Expanding

매일 수집한 데이터에는 노이즈가 있어서 일반적인 트렌드를 구하기 위해서 롤링민 / 무빙 평균을 사용한다.

롤링 함수를 이용해서 주어진 시한 내 평균등을 구할 수 있습니다.

임의 시간 간격의 window를 만들고 그 안에서 평균 등의 통계적 aggregation을 실행.

볼린저 밴드(Bollinger Band)

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [3]:
pycon_year = 2018
pycon_month = 8
pycon_day = 17
pycon_hour = 14
pycon_minute = 26
pycon_second = 20

In [4]:
pycon_date = datetime(pycon_year, pycon_month, pycon_day, pycon_hour, pycon_minute, pycon_second)

In [5]:
pycon_date

datetime.datetime(2018, 8, 17, 14, 26, 20)

In [6]:
main_dates = [datetime(2018, 8, 18), datetime(2019,8,19)]
main_dates

[datetime.datetime(2018, 8, 18, 0, 0), datetime.datetime(2019, 8, 19, 0, 0)]

In [7]:
dt_ind = pd.DatetimeIndex(main_dates)
dt_ind

DatetimeIndex(['2018-08-18', '2019-08-19'], dtype='datetime64[ns]', freq=None)

In [8]:
data = np.random.randn(2, 2)
print(data)

[[ 0.54819027  0.16623999]
 [-0.26834002 -0.71197827]]


In [9]:
df = pd.DataFrame(data, dt_ind, ['A','B'])
df

,A,B
2018-08-18,0.54819,0.166240
2019-08-19,-0.26834,-0.711978


In [10]:
df.index

DatetimeIndex(['2018-08-18', '2019-08-19'], dtype='datetime64[ns]', freq=None)

In [11]:
df.index.argmax()

1

### Time Resampling

이전에 Anaconda prompt 에서 Quandl 설치, Quandl API를 받아야 주가데이터를 받아올수있음

In [78]:
import quandl
quandl.ApiConfig.api_key = "4_QsK3sYLPGC7b6SCKQX"

start="2001-12-31"
end="2019-04-30"

In [79]:
data = quandl.get(["UMICH/SOC1", "MULTPL/SP500_PE_RATIO_MONTH"], start_date=start, end_date=end)
# University of Michigan Consumer Survey, Index of Consumer Sentiment
# S&P 500 PE Ratio by Month

In [80]:
data.head()

,UMICH/SOC1 - Index,MULTPL/SP500_PE_RATIO_MONTH - Value
Date,,
2001-12-31,88.8,NaN
2002-01-01,NaN,46.17
2002-01-31,93.0,NaN
2002-02-01,NaN,44.57
2002-02-28,90.7,NaN


In [81]:
data.tail()

,UMICH/SOC1 - Index,MULTPL/SP500_PE_RATIO_MONTH - Value
Date,,
2019-02-01,NaN,21.09
2019-02-28,93.8,NaN
2019-03-01,NaN,21.27
2019-03-31,98.4,NaN
2019-04-01,NaN,21.66


In [82]:
data.columns = ['Consumer_senti', 'SP500_PER']
data.to_excel('sampleData.xlsx')

In [83]:
data = pd.read_excel('sampleData.xlsx')

In [84]:
data.head()

,Date,Consumer_senti,SP500_PER
0,2001-12-31,88.8,NaN
1,2002-01-01,NaN,46.17
2,2002-01-31,93.0,NaN
3,2002-02-01,NaN,44.57
4,2002-02-28,90.7,NaN


In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 3 columns):
Date              416 non-null datetime64[ns]
Consumer_senti    208 non-null float64
SP500_PER         208 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 9.8 KB


In [86]:
data.set_index('Date', inplace = True)

In [87]:
data.head()

,Consumer_senti,SP500_PER
Date,,
2001-12-31,88.8,NaN
2002-01-01,NaN,46.17
2002-01-31,93.0,NaN
2002-02-01,NaN,44.57
2002-02-28,90.7,NaN


In [88]:
data.index.values

array(['2001-12-31T00:00:00.000000000', '2002-01-01T00:00:00.000000000',
       '2002-01-31T00:00:00.000000000', '2002-02-01T00:00:00.000000000',
       '2002-02-28T00:00:00.000000000', '2002-03-01T00:00:00.000000000',
       '2002-03-31T00:00:00.000000000', '2002-04-01T00:00:00.000000000',
       '2002-04-30T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-05-31T00:00:00.000000000', '2002-06-01T00:00:00.000000000',
       '2002-06-30T00:00:00.000000000', '2002-07-01T00:00:00.000000000',
       '2002-07-31T00:00:00.000000000', '2002-08-01T00:00:00.000000000',
       '2002-08-31T00:00:00.000000000', '2002-09-01T00:00:00.000000000',
       '2002-09-30T00:00:00.000000000', '2002-10-01T00:00:00.000000000',
       '2002-10-31T00:00:00.000000000', '2002-11-01T00:00:00.000000000',
       '2002-11-30T00:00:00.000000000', '2002-12-01T00:00:00.000000000',
       '2002-12-31T00:00:00.000000000', '2003-01-01T00:00:00.000000000',
       '2003-01-31T00:00:00.000000000', '2003-02-01

In [89]:
data.index[0]

Timestamp('2001-12-31 00:00:00')

In [90]:
data.index.month

Int64Index([12,  1,  1,  2,  2,  3,  3,  4,  4,  5,
            ...
            11, 12, 12,  1,  1,  2,  2,  3,  3,  4],
           dtype='int64', name='Date', length=416)

In [91]:
data.resample('Y').mean()

,Consumer_senti,SP500_PER
Date,,
2001-12-31,88.800000,NaN
2002-12-31,89.583333,37.277500
2003-12-31,87.625000,26.950833
2004-12-31,95.200000,20.500833
2005-12-31,88.550000,18.897500
2006-12-31,87.308333,17.294167
2007-12-31,85.583333,18.689167
2008-12-31,63.750000,28.390000
2009-12-31,66.258333,83.597500
